In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import time
from datetime import datetime


In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
}

In [3]:
articulos = []

for pag in range(1, 11):
    url = f'https://quehacerconlosninos.es/madrid/ocio/?e-page-499d516={pag}'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        titulos = soup.find_all('h3', class_='elementor-post__title')

        for t in titulos:
            titulo = t.get_text(strip=True)
            link = t.find('a')['href']

            articulos.append({
                'titulo': titulo,
                'url': link,
            })

            time.sleep(1.5)  # para no saturar el servidor

    else:
        print(f'❌ No se pudo acceder a la página {pag}')

In [4]:
articulos = []

for pag in range(1, 11):
    url = f'https://quehacerconlosninos.es/madrid/ocio/?e-page-499d516={pag}'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        titulos = soup.find_all('h3', class_='elementor-post__title')

        for t in titulos:
            titulo = t.get_text(strip=True)
            link = t.find('a')['href']

            try:
                # Entrar al artículo
                response_art = requests.get(link, headers=headers, timeout=10)
                soup_art = BeautifulSoup(response_art.text, 'html.parser')
                article = soup_art.find('div', class_='elementor-widget-theme-post-content')

                if article:
                    parrafos = article.find_all('p')
                    texto = " ".join(p.get_text(strip=True) for p in parrafos)

                    # Solo guardar si hay contenido real
                    if texto.strip():
                        articulos.append({
                            'titulo': titulo,
                            'url': link,
                            'contenido': texto
                        })
                        print(f"✅ Extraído: {titulo}")
                    else:
                        print(f"⚠️ Contenido vacío: {titulo}")
                else:
                    print(f"⚠️ No se encontró contenido en: {link}")

            except Exception as e:
                print(f"❌ Error con {link}: {e}")

            time.sleep(1.0)  # para no saturar el servidor

    else:
        print(f'❌ No se pudo acceder a la página {pag}')

✅ Extraído: Planes gratis en Madrid con niños esta primavera: 12 ideas para disfrutar sin gastar
✅ Extraído: Qué hacer en el puente de mayo con niños en Madrid
✅ Extraído: Escapes rooms para niños en Madrid: aventuras, misterios y diversión en familia
✅ Extraído: Huevos de Pascua con niños en Madrid: ideas para pintar, esconder y pasarlo en grande
✅ Extraído: Underwater bubble show madrid: una aventura submarina para disfrutar en familia
✅ Extraído: Dinosaurios en LaFinca Grand Café: una expedición jurásica para disfrutar con los niños
✅ Extraído: Droneart show en Madrid: el espectáculo que hará volar tu imaginación está a punto de aterrizar
✅ Extraído: La ruta de las torrijas, tan dulce como divertida para los niños
✅ Extraído: Día del niño y de la niña 2025: los museos de Cera, Thyssen, Guggenheim y  Oceanogràfic se suman a esta edición
✅ Extraído: Día del niño y de la niña 2025: los museos de Cera, Thyssen, Guggenheim y  Oceanogràfic se suman a esta edición
✅ Extraído: Los Mundos de

In [65]:
df = pd.DataFrame(articulos)

In [7]:
df.to_csv('Articulos.csv',index=False)

In [32]:
pd.set_option('display.max_colwidth', None)

In [66]:
regex_fechas = r"(?:\b(?:lunes|martes|miércoles|jueves|viernes|sábado|domingo)\s*)?\b\d{1,2}\s+de\s+(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)(?:\s+de\s+\d{4})?"
regex_precios = r"\d+(?:[\.,]\d+)? ?€"
meses = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4,
    "mayo": 5, "junio": 6, "julio": 7, "agosto": 8,
    "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}

def extraer_fechas(texto):
    return re.findall(regex_fechas, texto, flags=re.IGNORECASE)

def extraer_precio(texto):
    if pd.isnull(texto):
        return "No especificado"
    
    texto = texto.lower()
    
    # Buscar si contiene la palabra "gratis"
    if "gratis" in texto:
        return "Gratis"
    
    # Buscar precios en euros como "15€", "6 €", "20 €", etc.
    precios = re.findall(regex_precios, texto)
    if precios:
        return precios[0].replace(" ", "")
    
    return "No especificado"

def corregir_texto(texto):
    # Insertar espacio entre letras y números
    texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)
    texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
    return texto

def extraer_fecha_contexto_basico(texto):
    if pd.isnull(texto):
        return None

    texto = texto.lower()
    patrones = re.findall(r"(\d{1,2}) de (\w+)(?: de (\d{4}))?", texto)

    if not patrones:
        return None

    for dia, mes_nombre, año in patrones:
        mes = meses.get(mes_nombre)
        if mes is None:
            continue  # Ignorar si el mes no es reconocido

        año = int(año) if año else datetime.today().year

        try:
            fecha = datetime(int(año), int(mes), int(dia))
            hoy = datetime.today()
            return {
                "fecha_normalizada": fecha.strftime("%Y-%m-%d"),
                "dia_semana": fecha.strftime("%A"),
                "mes": fecha.strftime("%B"),
                "año": fecha.year,
                "es_este_mes": fecha.month == hoy.month and fecha.year == hoy.year,
                "es_este_fin_de_semana": fecha.weekday() in [5, 6] and 0 <= (fecha - hoy).days <= 7
            }
        except ValueError:
            continue

    return None


In [67]:
df['contenido'] = df['contenido'].apply(corregir_texto)
df['fechas_contexto'] = df['contenido'].apply(extraer_fecha_contexto_basico)
df['fechas'] = df['contenido'].apply(extraer_fechas)
df['Precios'] = df['contenido'].apply(extraer_precio)

In [68]:
df.head()

,titulo,url,contenido,fechas_contexto,fechas,Precios
0,Underwater bubble show madrid: una aventura submarina para disfrutar en familia,https://quehacerconlosninos.es/underwater-bubble-show-madrid-una-aventura-submarina-para-disfrutar-en-familia/,"¿Buscas unespectáculo familiar en Madridque sorprenda a grandes y pequeños? ElUnderwater Bubble Show Madridllega por primera vez a la ciudad para ofrecer una experiencia única llena de magia, burbujas y criaturas marinas. Una oportunidad perfecta para sumergirse en un mundo de fantasía y disfrutar en familia. ElUnderwater Bubble Showes un espectáculo internacional que ha conquistado a más de 10 millones de espectadores en 32 países. Inspirado en clásicos comoAlicia en el País de las MaravillasyPeter Pan, sigue la historia de Mr.B, un hombre moderno que, tras un día agitado, es transportado mágicamente aBubblelandia, un mundo submarino repleto de fantasía, burbujas y criaturas sorprendentes como caballitos de mar, peces dragón, sirenas y estrellas de mar. Este espectáculo combinaefectos visuales, música envolvente, acrobacias y, por supuesto, una impresionante variedad de burbujas de todos los tamaños y colores. La puesta en escena está diseñada para estimular la imaginación y los sentidos de los espectadores, creando una atmósfera inmersiva que transporta al público al fondo del mar. Es una experiencia que deleita tanto a niños como a adultos, ofreciendo diferentes niveles de interpretación y disfrute. Los personajes deBubblelandiaestán cuidadosamente diseñados para conectar con el público de todas las edades. Desde las encantadoras sirenas hasta los juguetones peces dragón, cada criatura aporta una dimensión única al espectáculo. La escenografía, llena decolores vibrantes y efectos especiales, crea un ambiente mágico que envuelve a los espectadores y los hace sentir parte de la aventura submarina. Más allá del entretenimiento, elUnderwater Bubble Showtransmite mensajes profundos sobre la importancia de la imaginación, la desconexión del estrés diario y la reconexión con nuestro niño interior. Invita a reflexionar sobre la necesidad de detenerse, soñar y redescubrir la magia en lo cotidiano. Estos valores educativos se presentan de forma lúdica y accesible, facilitando conversaciones significativas entre padres e hijos después del espectáculo. ElUnderwater Bubble Showse presentará en Madrid elsábado 3 de mayo de 2025 a las 19:00 horasen elPalacio Vistalegre, ubicado en la Calle de Utebo, 1, 28025 Madrid, Carabanchel. Las entradas están disponibles en diferentes rangos de precios, entre 33 y 55 euros, y se pueden adquirir a través de lapágina oficial del espectáculo. Se recomienda llegar con antelación para evitar aglomeraciones y disfrutar de las actividades previas al show. Además, el recinto cuenta con accesos adaptados para personas con movilidad reducida y ofrece facilidades para familias con niños pequeños, como zonas de descanso y servicios adecuados. Síguenos en nuestrocanal de WhatsApppara recibir más planes y noticias en tiempo real.","{'fecha_normalizada': '2025-05-03', 'dia_semana': 'Saturday', 'mes': 'May', 'año': 2025, 'es_este_mes': False, 'es_este_fin_de_semana': False}",[3 de mayo de 2025],No especificado
1,Droneart show en Madrid: el espectáculo que hará volar tu imaginación está a punto de aterrizar,https://quehacerconlosninos.es/droneart-show-en-madrid/,"La cuenta atrás ha comenzado: en solo unos días se abrirá la lista de espera para elDroneArt Show en Madrid, un evento quepromete convertirse en uno de los planes más sorprendentes del año. Música clásica en directo, un cielo pintado con drones y miles de velas crean una atmósfera mágica que dejará boquiabiertos a pequeños y mayores. Después de arrasar en ciudades como Miami y Los Ángeles, elDroneArt Showaterriza por primera vez en España.El Hipódromo de la Zarzuela será el escenario donde se unirá lo clásico con lo futurista, en una experiencia sensorial para disfrutar en familia. Durante más de una hora,un talento

In [71]:
df.to_csv('Articulos_fechas.csv',index=False)